# Inverse Kinematics Optimization

The previous doc explained features and how they define objectives of a constrained optimization problem. Here we show how to use this to solve IK optimization problems.

At the bottom there is more general text explaining the basic concepts.

## Demo of features in Inverse Kinematics

Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path.append('../rai/rai/ry')
import numpy as np
import libry as ry

In [2]:
C = ry.Config()
D = C.view()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../test/kitchen.g')

For simplicity, let's add a frame that represents goals

In [3]:
goal = C.addFrame("goal")
goal.setShape(ry.ST.sphere, [.05])
goal.setColor([.5,1,1])
goal.setPosition([1,.5,1])
X0 = C.getFrameState() #store the initial configuration

We create an IK engine. The only objective is that the `positionDiff` (position difference in world coordinates) between `pr2L` (the yellow blob in the left hand) and `goal` is equal to zero:

In [4]:
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'])

We now call the optimizer (True means with random initialization/restart).

In [5]:
IK.optimize(True)
IK.getReport()

[{'T': 1,
  'X': [0.0,
   0.0,
   0.0,
   1.0,
   0.0,
   0.0,
   0.0,
   0.12163903227044062,
   0.03048345913810413,
   0.0,
   0.9998847606641896,
   0.0,
   0.0,
   -0.015181086638189229,
   0.07166207880959227,
   0.032001392856088946,
   0.8906752479378529,
   0.7070252946707365,
   -0.01073464930764409,
   -0.7070252946707365,
   -0.01073464930764409,
   0.054599946898058756,
   0.032519615427408964,
   1.2721252479378529,
   0.7070125101015544,
   -0.011546019223059412,
   -0.7070125101015544,
   -0.011546019223059412,
   0.17054647892722694,
   0.02899800920168419,
   1.117675247937853,
   -0.7177598930809153,
   0.00027030412721090996,
   0.0002622189170055871,
   -0.6962905959880481,
   0.06595464802996936,
   -0.15591195215670098,
   0.8906752479378529,
   0.6169120627682251,
   -0.34557127602139226,
   -0.6169120627682251,
   -0.34557127602139226,
   0.07736950958921518,
   0.21991473786887886,
   0.8906752479378529,
   0.6766435309368731,
   0.2053132534477029,
   -0.6766

The best way to retrieve the result is to copy the optimized IK configuration back into your working configuration C, which is now also displayed

In [6]:
C.setFrameState( IK.getConfiguration(0) )

We can redo the optimization, but for a different configuration, namely a configuration where the goal is in another location.
For this we move goal in our working configuration C, then copy C back into the IK engine's configurations:

In [13]:
## (iterate executing this cell for different goal locations!)

# move goal
goal.setPosition([.8,.2,1.5])

# copy C into the IK's internal configuration(s)
IK.setConfigurations(C)

# reoptimize
IK.optimize(False) # False indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getConfiguration(0) )

[{'x_dim': 25, 'T': 1, 'k_order': 1, 'tau': 1.0, 'useSwift': False, 'X': [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.7780912334369787, -0.6036739325926503, 0.0, -0.9965221022315975, -0.0, 0.0, 0.08332886513038187, 0.7287856034133705, -0.5953700270070204, 0.8907099202736581, 0.7046475360902367, -0.05892240560227227, -0.7046475360902367, -0.05892240560227227, 0.7119526613233107, -0.5925350736400863, 1.2721599202736589, -0.7045785928399059, 0.059741162624594876, 0.7045785928399059, 0.059741162624594876, 0.8263417229780822, -0.6118001345987482, 1.1177099202736582, 0.7635698788873572, -0.00030969702624622, -0.00026190024222436044, 0.6457250773757838, 0.6975629184114012, -0.7807591958957876, 0.8907099202736581, -0.5918712423770596, 0.3868958935515289, 0.5918712423770596, 0.3868958935515289, 0.7600082884153398, -0.40998085811825313, 0.8907099202736581, -0.48488042842574064, 0.5146755969828667, 0.48488042842574064, 0.5146755969828667, 0.7789818915521179, -0.603984191288777, 1.2721599202736589, -0.74

Let's solve some other problems, always creating a novel IK engine:

The relative position of `goal` in `pr2R` coordinates equals [0,0,-.2] (which is 20cm straight in front of the yellow blob)

In [14]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, feature=ry.FS.positionRel, frames=['goal','pr2R'], target=[0,0,-.2])
IK.optimize(True)
C.setFrameState( IK.getConfiguration(0) )

The distance between `pr2R` and `pr2L` is zero:

In [15]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, feature=ry.FS.distance, frames=['pr2L','pr2R'])
IK.optimize(True)
C.setFrameState( IK.getConfiguration(0) )

The 3D difference between the z-vector of `pr2R` and the z-vector of `goal`:

In [16]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, feature=ry.FS.vectorZDiff, frames=['pr2R', 'goal'])
IK.optimize(True)
C.setFrameState( IK.getConfiguration(0) )

The scalar product between the z-vector of `pr2R` and the z-vector of `goal` is zero:

In [17]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['pr2R', 'goal'])
IK.optimize(True)
C.setFrameState( IK.getConfiguration(0) )

etc etc

## More explanations

All methods to compute paths or configurations solve constrained optimization problems. To use them, you need to learn to define constrained optimization problems. Some definitions:

* An objective defines either a sum-of-square cost term, or an equality constraint, or an inequality constraint in the optimization problem. An objective is defined by its type and its feature. The type can be `sos` (sum-of-squares), `eq`, or `ineq`, referring to the three cases.
* A feature is a (differentiable mapping) from the decision variable (the full path, or robot configuration) to a feature space. If the feature space is, e.g., 3-dimensional, this defines 3 sum-of-squares terms, or 3 inequality, or 3 equality constraints, one for each dimension. For instance, the feature can be the 3-dim robot hand position in the 15th time slice of a path optimization problem. If you put an equality constraint on this feature, then this adds 3 equality constraints to the overall path optimization problem.
* A feature is defined by the keyword for the feature map (e.g., `pos` for position), typically by a set of frame names that tell which objects we refer to (e.g., `pr2L` for the left hand of the pr2), optionally some modifiers (e.g., a scale or target, which linearly transform the feature map), and the set of configuration IDs or time slices the feature is to be computed from (e.g., `confs=[15]` if the feat is to be computed from the 15th time slice in a path optimization problem).
* In path optimization problems, we often want to add objectives for a whole time interval rather than for a single time slice or specific configuration. E.g., avoid collisions from start to end. When adding objectives to the optimization problem we can specify whole intervals, with `times=[1., 2.]`, so that the objective is added to each configuration in this time interval.
* Some features, especially velocity and acceleration, refer to a tuple of (consecutive) configurations. E.g., when you impose an acceleration feature, you need to specify `confs=[13, 14, 15]`. Or if you use `times=[1.,1.]`, the acceleration features is computed from the configuration that corresponds to time=1 and the two configurations *before*.
* All kinematic feature maps (that depend on only one configuration) can be modified to become a velocity or acceleration features. E.g., the position feature map can be modified to become a velocity or acceleration feature.
* The `sos`, `eq`, and `ineq` always refer to the feature map to be *zero*, e.g., constraining all features to be equal to zero with `eq`. This is natural for many features, esp. when they refer to differences (e.g. `posDiff` or `posRel`, which compute the relative position between two frames). However, when one aims to constrain the feature to a non-zero constant value, one can modify the objective with a `target` specification.
* Finally, all features can be rescaled with a `scale` specification. Rescaling changes the costs that arise from `sos` objectives. Rescaling also has significant impact on the convergence behavior for `eq` and `ineq` constraints. As a guide: scale constraints so that if they *would* be treated as squared penalties (squaredPenalty optim mode, to be made accessible) convergence to reasonable approximate solutions is efficient. Then the AugLag will also converge efficiently to precise constraints.

In [18]:
# Designing a cylinder grasp

In [19]:
D=0
C=0

In [20]:
C = ry.Config()
D = C.view()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../test/kitchen.g')
C.setJointState([.7], ["l_gripper_l_finger_joint"])

In [21]:
goal = C.addFrame("goal")
goal.setShape(ry.ST.cylinder, [0,0,.2, .03])
goal.setColor([.5,1,1])
goal.setPosition([1,.5,1])
X0 = C.getFrameState()

In [22]:
C.setFrameState(X0)
goal.setPosition([1,.5,1.2])
IK = C.komo_IK()
IK.addObjective(type=ry.OT.eq,   feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scaleTrans=[[1,0,0],[0,1,0]])
IK.addObjective(type=ry.OT.ineq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scaleTrans=[[0,0,1]], target=[0,0,.05])
IK.addObjective(type=ry.OT.ineq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scaleTrans=[[0,0,-1]], target=[0,0,-.05])
IK.addObjective(type=ry.OT.sos,  feature=FS.scalarProductZZ, frames=['pr2L', 'goal'], scale=[0.1])
IK.addObjective(type=ry.OT.eq,   feature=FS.scalarProductXZ, frames=['pr2L', 'goal'])
IK.optimize()
K.setFrameState( IK.getConfiguration(0) )
IK.getReport()

TypeError: komo_IK(): incompatible function arguments. The following argument types are supported:
    1. (self: libry.Config, useSwift: bool) -> ry::RyKOMO

Invoked with: <libry.Config object at 0x7f0b0ee0f110>